## 0. Libraries and version

In [322]:
from sys import version 
import IPython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

import warnings
warnings.filterwarnings('ignore')

print('Python version:     ' + version)
print('Numpy version:      ' + np.__version__)
print('IPython version:    ' + IPython.__version__)
print('Pandas version:     ' + pd.__version__)
print('SciPy version:      ' + scipy.__version__)

Python version:     3.10.13 (main, Sep 11 2023, 08:24:56) [Clang 14.0.6 ]
Numpy version:      1.26.4
IPython version:    8.16.1
Pandas version:     2.0.3
SciPy version:      1.10.1


## 1. Introduction to Asian options and their valuation (arithmetric averaging, fixed strike)

The Asian option is a path-dependent exotic option whose payoff involves a historical average price of the underlying asset.

Due to the averaging mechanism, Asian options have lower volatility and offers greater protection against price fluctuations compared to the plain European counterparts and are prevalent in the commodities, currency and energy markets.

We begin with the geometric Brownian motion (GBM) stock process, $S(t)$ whose dynamics are given by
$$
\begin{equation}
    d S(t)=r S(t) d t+\sigma S(t) d \widetilde{W}(t)
\end{equation}
$$
where $\widetilde{W}(t), 0\leq t\leq T$ denotes the Brownian motion under the risk neutral measure $\widetilde{\mathbb{P}}$.

The payoff of the Asian call with the non-negative fixed-strike $K$ under arithmetic averaging at time $T$ is given by

$$V(T) = \left(\frac{1}{T} \int_0^T S(t) d t-K\right)^{+}$$

Assuming a constant interest rate $r$, the price for $t \leq T$ is given by the risk-neutral pricing formula 

$$ V(t)=\tilde{\mathbb{E}}\left[e^{-r(T-t)} V(T) \mid \mathcal{F}(t)\right], \quad 0 \leq t \leq T$$

Under the Black-Scholes model, the option price is expressed in terms of both $S(t)$ and auxiliary variable $A(t) = \int_0^t S(u) d u$ to obtain the Asian call option PDE:
$$ \frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 A^2 \frac{\partial^2 V}{\partial A^2} + r A \frac{\partial V}{\partial A} - r V = 0$$

The stock process is given by $S \sim \text{GBM} (\mu, \sigma^2)$, i.e.

$$dS_t = \mu S_t dt + \sigma S_t dW$$

where $W$ is a Brownian motion. The solution to the PDE is given by

## 2. Stock process

### 2.1. Black-Scholes model

The class `GBM` below implements the stock process (geometric Brownian motion) as in (1).

In [ ]:
from scipy.stats import norm

class GBM():

    # Initialise GBM model
    def __init__(self, S_0, r, sigma, T, n):
        self.S_0 = S_0        
        self.r = r          
        self.sigma = sigma
        self.T = T
        self.n = n
        self.dt = T/(n-1)

    # Generate N paths
    def generate_paths(self, n_paths):

        X_0 = np.zeros((n_paths, 1))
        r, sigma, dt = self.r, self.sigma, self.dt

        # Create normal increments in the exponent
        dW = norm.rvs(loc = (r - sigma**2 / 2) * dt,
                      scale = np.sqrt(dt) * sigma,
                      size = (n_paths, self.n - 1))

        # Multiply by spot price
        X = np.concatenate((X_0, dW), axis=1).cumsum(1)

        return self.S_0 * np.exp(X)
    
    def __str__(self):

        S_0, r, sigma, T, n = self.S_0, self.r, self.sigma, self.T, self.n

        return f'GBM with S_0 = {S_0}, r = {r}, sigma = {sigma}, T = {T}, n (discretisation) = {n}'

In [276]:
np.random.seed(0)

S_0 = 100.0  # spot stock price
T = 1  # maturity
r = 0.1  # risk free rate
sigma = 0.2  # diffusion coefficient or volatility

n = 10000 # number of discretisation points

S = GBM(S_0, r, sigma, T, n)

## 3. Option Pricing Methods

### 3.1 Laplace inversion method 

#### Talbot algorithm

In [372]:
from scipy import integrate
from scipy.special import gamma

K = 100 # Strike price
M = 100 # Parameters of the algorithm

S_0, T, r, sigma = S.S_0, S.T, S.r, S.sigma

h = sigma**2 / 4
q = K * T * sigma**2 / (4 * S_0)
nu = 2*r / sigma**2 - 1

d = np.zeros(M, dtype=complex)
d[0] = 2 * M / 5
for k in range(1, M):
    d[k] = 2 * k * np.pi / 5 * (1 / np.tan(k * np.pi / M) + 1j)

g = np.zeros(M, dtype=complex)
g[0] = 0.5 * np.exp(d[0])
for k in range(1, M):
    g[k] = (1 + 1j * (k * np.pi / M) * (1 + (1 / np.tan(k * np.pi / M))**2) - 1j * (1 / np.tan(k * np.pi / M))) * np.exp(d[k])

arg = d / h

if np.any(np.abs(arg) < np.maximum(0, 2 * (nu + 1))):
    raise ValueError('The argument not in right half-plane.')

mu = np.sqrt(2 * arg + nu**2)
alpha = (mu + nu) / 2
beta = (mu - nu) / 2

def complex_quadrature(func, a, b):
    def real_func(x): return np.real(func(x))
    def imag_func(x): return np.imag(func(x))

    real_integral = integrate.quad(real_func, a, b)
    imag_integral = integrate.quad(imag_func, a, b)
    return real_integral[0] + 1j*imag_integral[0]

def integrand(u, alpha, beta, q):
    return u**(beta - 2) * (1 - u)**(alpha + 1) * np.exp(-u / (2 * q))

# Handle arrays (integrate over multiple values of alpha, beta)
results = []
for a, b in zip(alpha, beta):
    results.append(complex_quadrature(lambda u: integrand(u, a, b, q), 0.001, 0.999))

results = np.array(results) 

res = (2 * q)**(1 - beta) / (2 * arg * (alpha + 1) * gamma(beta)) * results * g
res = np.real(res)
res =  2 / (5 * h) * np.sum(res)

res *= np.exp(-r * T) * (S_0 * 4) / (T * sigma**2)
res

7.040916033316196

In [373]:
# K is the strike price
def MC_asian(GBM, trials=1000, K=100):

    paths = GBM.generate_paths(trials)

    T = GBM.T
    
    # Get average of each path
    A = 1/T * np.mean(paths, axis=1)

    # Get payoff of each path
    V = np.maximum(A-K, 0)

    # Get average of the result
    res = np.exp(-r*T) * np.mean(V)

    # Get standard error
    err = np.exp(-r*T) * scipy.stats.sem(V)

    return err, res

err, res = MC_asian(S, 5000, K)

print(S)
print(f'(Basic) Monte Carlo price: {res:.5f} with standard error {err:.5f}')

GBM model with S_0 = 100.0, r = 0.1, sigma = 0.2, T = 1, n (discretisation) = 10000
(Basic) Monte Carlo price: 7.23509 with standard error 0.12285


### Discrete Geometric Asian options

The discrete geometric Asian option with strike price $K$ and expiry $T$ and $N$ monitoring dates is given by

$$ C_0 = \mathrm{e}^{-r T} \mathbb{E} \left( G_N - K \right)^+, \quad G_N = \left( \prod_{i=1}^N S(t_i) \right)^{1/N} $$

which has the closed form solution

$$ C_0 = \mathrm{e}^{-r T}\left(S_0 \exp \left(c_N\right) \Phi\left(d_N+\sigma b_N \sqrt{T}\right)-K \Phi\left(d_N\right)\right) $$

where 

$$ c_N=\mu a_N T+\frac{\left(\sigma b_N \sqrt{T}\right)^2}{2}, \quad d_N=\frac{-\log \left(K / S_0\right)+\mu a_N T}{\sigma b_N \sqrt{T}}$$

$$ a_N=\frac{N+1}{2 N}, \quad b_N=\sqrt{\frac{(N+1)(2N+1)}{6N^2}}$$


In [374]:
def discrete_price(GBM, K, N=1000):
    
    S_0, T, r, sigma = GBM.S_0, GBM.T, GBM.r, GBM.sigma

    mu = (r - 0.5 * sigma**2)

    a_N = (N+1) / (2*N)
    b_N = np.sqrt((N+1) * (2*N+1) / (6*N**2))

    c_N = mu * a_N * T + 0.5 * (sigma * b_N * np.sqrt(T)) ** 2 
    d_N = (-np.log(K/S_0) + mu * a_N * T ) / (sigma * b_N * np.sqrt(T))

    # price = np.exp(-r * T) * (S_0 * np.exp(c_N) * norm.cdf(d_N + sigma * b_N * np.sqrt(T)) - K * norm.cdf(d_N))
    price = (S_0 * np.exp(c_N) * norm.cdf(d_N + sigma * b_N * np.sqrt(T)) - K * norm.cdf(d_N))

    return price

np.exp(-r * T) * discrete_price(S, K)

6.7761085175159135

### Continuous Geometric Asian options

The continuous geometric Asian option with strike price $K$ and expiry $T$ is given by

$$C_0 = \mathrm{e}^{-r T} \mathbb{E} \left( G_T - K \right)^+, \quad G_T = \exp \left( \frac{1}{T} \int_0^T \log S(t) d t \right) $$

which has the closed form solution

$$ C_0 = \mathrm{e}^{-r T}\left(S_0 \exp \left(c\right) \Phi\left(d+ \frac{1}{\sqrt{3}} \sigma  \sqrt{T}\right)-K \Phi\left(d\right)\right) $$

where 

$$ c = \frac{1}{2} \mu T+\frac{1}{6}(\sigma \sqrt{T})^2, \quad d =\sqrt{3} \frac{\log \left( \frac{S_0}{K}\right)+ \frac{\mu T}{2}}{\sigma \sqrt{T}}$$


In [312]:
def continuous_price(GBM, K):
    
    S_0, T, r, sigma = GBM.S_0, GBM.T, GBM.r, GBM.sigma

    mu = (r - 0.5 * sigma**2)

    c = 0.5 * mu * T + 1/6 * sigma**2 * T
    d = np.sqrt(3) * (-np.log(K/S_0) + 0.5 * mu * T ) / (sigma * np.sqrt(T))

    # price = np.exp(-r * T) * (S_0 * np.exp(c) * norm.cdf(d + np.sqrt(T / 3) * sigma) - K * norm.cdf(d))
    price = (S_0 * np.exp(c) * norm.cdf(d + np.sqrt(T / 3) * sigma) - K * norm.cdf(d))

    return price

np.exp(-r*T) * continuous_price(S, K)

6.769950595122837

## Control Variate Monte Carlo

In [375]:
# K is the strike price
def CVMC_asian(GBM, trials=1000, K=100):

    paths = GBM.generate_paths(trials)

    r, T = GBM.r, GBM.T
    
    # Get arithmetic payoff
    arith_payoff = np.maximum(1/T * np.mean(paths, axis=1) - K, 0)

    #### Use Geometric asian option price as CV ####

    # Option 1. Continuous geometric asian option price
    geo_payoff = np.maximum(np.exp(1/T * np.mean(np.log(paths), axis=1)) - K, 0)
    geo_price = continuous_price(GBM, K)

    # # Option 2. Discrete geometric asian option price
    # N = GBM.n - 1
    # geo_payoff = np.maximum(np.exp(1/N * np.sum(np.log(paths), axis=1)) - K, 0)
    # geo_price = discrete_price(GBM, K, N)

    ################################################

    # Get covariance of the two payoffs
    cov = np.mean(arith_payoff * geo_payoff) - np.mean(arith_payoff) * np.mean(geo_payoff)

    # Get the beta coefficient
    beta = cov / np.var(geo_payoff)

    # Get control variate estimator
    CV_estimator = arith_payoff + beta * (geo_price - geo_payoff)

    res = np.exp(-r*T) * np.mean(CV_estimator)
    err = np.exp(-r*T) * scipy.stats.sem(CV_estimator)

    return err, res

err, res = CVMC_asian(S, 10000, K)

print(S)
print(f'Control variate MC price (K={K}): {res:.5f} with standard error {err:.5f}')


GBM model with S_0 = 100.0, r = 0.1, sigma = 0.2, T = 1, n (discretisation) = 10000
Control variate MC price (K=100): 7.03735 with standard error 0.00245
